In [27]:

# 1. Defining a function that takes params and return result+params

import import_ipynb
from strategy_base import TradeStrategyBase
    
def execute(strategy: TradeStrategyBase,
         keep_stock_threshold: int, 
         buy_change_threshold: float):
    """
    :param keep_stock_threshold: 
    :param buy_change_threshold: 
    :return:  
    """
    
    # Use class method to set keep_stock_threshold
    strategy.set_keep_stock_threshold(keep_stock_threshold)
    
    # Use static method to set buy_change_threshold
    strategy.set_buy_change_threshold(buy_change_threshold)
    
    # Create Trade Days (MasterCard)
    from abupy import ABuSymbolPd, EMarketDataSplitMode
    # List 2 years of MasterCard closing price to list()
    price_array = ABuSymbolPd.make_kl_df('MA', EMarketDataSplitMode.E_DATA_SPLIT_SE).close.tolist()
    date_array = ABuSymbolPd.make_kl_df('MA', EMarketDataSplitMode.E_DATA_SPLIT_SE).date.tolist()
    price_array[-5:], date_array[-5:]
    
    from tradedays import StockTradeDays
    trade_days = StockTradeDays(price_array, "", date_array)
    # print('trade_days has {} days'.format(len(trade_days)))
    # print('Last trading day: {}'.format(trade_days[-1]))

    # Backtest
    from trade_loopback import TradeLoopBack
    trade_loop_back = TradeLoopBack(trade_days, strategy)
    trade_loop_back.execute_trade()
    
    # Calculate profit after back test
    from functools import reduce
    profit = 0.0 if len(trade_loop_back.profit_array) == 0 else \
        reduce(lambda a, b: a + b, trade_loop_back.profit_array)
    
    # Return profit and the two input params
    return profit, keep_stock_threshold, buy_change_threshold


In [28]:

# Initialize an instance of Strategy2
import import_ipynb
from strategy_2 import TradeStrategy2
trade_strategy2 = TradeStrategy2()

execute(trade_strategy2, 5, -0.02)


(0.1370000000000001, 5, -0.02)

In [29]:

# range: buy and then hold 2~30 days, step is 2
keep_stock_days_list = range(2, 30, 2)
print('keep_stock Array: {}'.format(list(keep_stock_days_list)))
# range: 'buy_change' from -0.05 to -0.15, aka down 5% to 15%
buy_change_list = [buy_change / 100.0 for buy_change in range(-5, -16, -1)]
print('buy_change Array: {}'.format(buy_change_list))


keep_stock Array: [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28]
buy_change Array: [-0.05, -0.06, -0.07, -0.08, -0.09, -0.1, -0.11, -0.12, -0.13, -0.14, -0.15]


In [31]:

# Due go GIL (Global Interpreter Lock), Python can only execute one thread at a time. 
# So Python multi-threading works the best for IO-Bound and Concurrent Tasks for blocking operations, or multi processor concurrent CPU-Bound tasks.

# 1. Using ProcessPollExecutor

result = []
# Callback function
def done_callback(r):
    # done_callback runs on the main process
    result.append(r.result())

from concurrent.futures.process import ProcessPoolExecutor
import itertools

with ProcessPoolExecutor() as pool:
    for keep_stock_threshold, buy_change_threshold in itertools.product(keep_stock_days_list, buy_change_list):
        """
        Submit task, use calc() to submit
        """
        future_result = pool.submit(execute, 
                                    trade_strategy2, 
                                    keep_stock_threshold, 
                                    buy_change_threshold)
        future_result.add_done_callback(done_callback)
        
sorted(result)[::-1][:10]

[(0.37900000000000006, 8, -0.05),
 (0.306, 6, -0.05),
 (0.29800000000000004, 20, -0.05),
 (0.29300000000000004, 12, -0.05),
 (0.292, 8, -0.06),
 (0.289, 10, -0.05),
 (0.2749999999999999, 28, -0.05),
 (0.2719999999999999, 26, -0.05),
 (0.263, 16, -0.05),
 (0.2560000000000001, 18, -0.05)]

In [32]:

# 2. Using ThreadPollExecutor
result = []
# Callback function
def done_callback(r):
    # done_callback runs on the main process
    result.append(r.result())
    
from concurrent.futures import ThreadPoolExecutor
import itertools

with ThreadPoolExecutor() as pool:
    for keep_stock_threshold, buy_change_threshold in itertools.product(keep_stock_days_list, buy_change_list):
        future_result = pool.submit(execute, 
                                    trade_strategy2, 
                                    keep_stock_threshold,
                                    buy_change_threshold)
        future_result.add_done_callback(done_callback)
        
sorted(result)[::-1][:10]

[(0.29800000000000004, 12, -0.14),
 (0.29300000000000004, 6, -0.13),
 (0.289, 4, -0.05),
 (0.2749999999999999, 26, -0.1),
 (0.2719999999999999, 18, -0.07),
 (0.263, 12, -0.1),
 (0.2560000000000001, 6, -0.15),
 (0.2399999999999999, 14, -0.12),
 (0.21100000000000002, 2, -0.08),
 (0.18800000000000003, 2, -0.1)]

In [ ]:

# 3. 'with' and unmanaged resources
def __enter__(self):
    self.acquire()
def __exit__(self, t, v, tb):
    self.release()